model created corona tweet 98% accuracy epcohs->6
LEFT:->creating an app left, model save left

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

In [ ]:
train=pd.read_csv(r'/content/drive/MyDrive/datasets/corona tweet/Corona_NLP_train.csv',encoding='latin-1')
test=pd.read_csv(r'/content/drive/MyDrive/datasets/corona tweet/Corona_NLP_test.csv',encoding='latin-1')

In [ ]:
train.shape

(41157, 6)

In [ ]:
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
train.Sentiment.value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [ ]:
train.drop(['UserName','ScreenName','Location','TweetAt'],axis=1,inplace=True)

In [ ]:
train.reset_index(inplace=True)

In [ ]:
train.iloc[1105]

index                                                         1105
OriginalTweet    Interviewed on Bill Kelly show. See podcast be...
Sentiment                                                  Neutral
Name: 1105, dtype: object

In [ ]:
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import seaborn as sns

In [ ]:
MAX_LEN=128

In [ ]:
dict_map={'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2}
train['Sentiment']=train['Sentiment'].map(dict_map)

In [ ]:
train.head()

,index,OriginalTweet,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,1,advice Talk to your neighbours family to excha...,2
2,2,Coronavirus Australia: Woolworths to give elde...,2
3,3,My food stock is not the only one which is emp...,2
4,4,"Me, ready to go at supermarket during the #COV...",0


In [ ]:
X=train['OriginalTweet'].values
y=to_categorical(train['Sentiment'],dtype='int32')

In [ ]:
type(X)

numpy.ndarray

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=12,shuffle=True)

In [ ]:
X_train=X_train.tolist()
X_test=X_test.tolist()

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize(data):
  input_ids=[]
  attention_masks=[]
  for i in range(len(data)):
    encoded=tokenizer.encode_plus(
            data[i],
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
  return input_ids,attention_masks



In [ ]:
train_input_ids, train_attention_masks = tokenize(X_train)
val_input_ids, val_attention_masks = tokenize(X_test)


In [ ]:
for i in train_input_ids:
  if(len(i)!=128):
    print(len(i))

In [ ]:
np.asarray(train_input_ids)

array([[ 101, 2182, 2024, ...,    0,    0,    0],
       [ 101, 2023, 2003, ...,    0,    0,    0],
       [ 101, 1030, 3198, ...,    0,    0,    0],
       ...,
       [ 101, 1029, 1029, ...,    0,    0,    0],
       [ 101, 1001, 2522, ...,    0,    0,    0],
       [ 101, 2307, 2466, ...,    0,    0,    0]])

In [ ]:
train_attention_masks,train_input_ids=np.asarray(train_attention_masks),np.asarray(train_input_ids)
val_attention_masks,val_input_ids=np.asarray(val_attention_masks),np.asarray(val_input_ids)

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
embedding=bert_model([train_input_ids,train_attention_masks])

In [ ]:
embedding.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
#for 3 inputs
def create_model(bert_model,max_len=MAX_LEN):
  loss=tf.keras.losses.CategoricalCrossentropy()
  accuracy=tf.keras.metrics.CategoricalAccuracy()

  input_ids=tf.keras.Input(shape=(max_len,),dtype='int32')
  attention_masks=tf.keras.Input(shape=(max_len,),dtype='int32')
  embedding=bert_model([input_ids,attention_masks])[1]
  output=tf.keras.layers.Dense(3,activation='softmax')(embedding)
  model=tf.keras.models.Model(inputs=[input_ids,attention_masks],outputs=output)
  model.compile('adam',loss='categorical_crossentropy',metrics=accuracy)
  return model

In [ ]:
from tensorflow.keras.layers import Dense,Dropout
import tensorflow as tf

In [ ]:
def create_model(bert_model,max_len=MAX_LEN):
  # opt=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-7)
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  accuracy=tf.keras.metrics.CategoricalAccuracy()
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

  input_ids=tf.keras.Input(shape=(max_len,),dtype='int32',name='layer_input_ids')
  attention_masks=tf.keras.Input(shape=(max_len,),dtype='int32',name='layer_attention_mask')

  embedding=bert_model([input_ids,attention_masks])[1]
  embedding=tf.keras.layers.Dense(128)(embedding)
  output=tf.keras.layers.Dense(3,activation='softmax')(embedding)

  model=tf.keras.models.Model(inputs=[input_ids,attention_masks],outputs=output)
  model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

  return model

In [ ]:
input_train_data={
    'layer_input_ids':train_input_ids,
    'layer_attention_mask':train_attention_masks,

}
input_test_data={
    'layer_input_ids':val_input_ids,
    'layer_attention_mask':val_attention_masks,
}


In [ ]:
model=create_model(bert_model,MAX_LEN)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 layer_input_ids (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 layer_attention_mask (Inpu  [(None, 128)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['layer_input_ids[0][0]',     
 )                           ngAndCrossAttentions(last_   40         'layer_attention_mask[0

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((input_train_data, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((input_test_data, y_test))


In [ ]:
batch_size=32

In [ ]:
history=model.fit(train_dataset.batch(batch_size), epochs=3)

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1029/1029 [==============================] - 873s 794ms/step - loss: 0.5732 - accuracy: 0.7638
Epoch 2/3
1029/1029 [==============================] - 820s 797ms/step - loss: 0.2944 - accuracy: 0.8985
Epoch 3/3
1029/1029 [==============================] - 821s 798ms/step - loss: 0.1954 - accuracy: 0.9354


In [ ]:
history=model.fit(train_dataset.batch(batch_size), epochs=6)

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1029/1029 [==============================] - 874s 800ms/step - loss: 0.1552 - accuracy: 0.9482
Epoch 2/6
1029/1029 [==============================] - 823s 800ms/step - loss: 0.1089 - accuracy: 0.9637
Epoch 3/6
1029/1029 [==============================] - 822s 799ms/step - loss: 0.0888 - accuracy: 0.9708
Epoch 4/6
1029/1029 [==============================] - 822s 799ms/step - loss: 0.0706 - accuracy: 0.9756
Epoch 5/6
1029/1029 [==============================] - 822s 799ms/step - loss: 0.0549 - accuracy: 0.9816
Epoch 6/6
1029/1029 [==============================] - 823s 799ms/step - loss: 0.0469 - accuracy: 0.9843


In [ ]:
model.evaluate(test_dataset.batch(batch_size))

258/258 [==============================] - 78s 286ms/step - loss: 0.3937 - accuracy: 0.9187


[0.3937308192253113, 0.9187317490577698]

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('/content/sentiment_model','wb'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('/content/sentiment_model1.hdf5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
test_data=test['OriginalTweet'].values

In [ ]:
test_data

array(['TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1',
       "When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY",
       'Find out how you can protect yourself and loved ones from #coronavirus. ?',
       ...,
       "Asst Prof of Economics @cconces was on @NBCPhiladelphia talking about her recent research on coronavirus' impact on the economy. Watch it here (starting at :33): https://t.co/8tfYNoro5l",
       "Gov need to do somethings instead of biar je rakyat assume 'lockdown' ke or even worst. Harini semua supermarket crowded like hell. Lagi mudah virus tu tersebar ?? #COVID2019",
       'I and @ForestandPaper members are committed to the safety of our employe

In [ ]:
model.predict(test_data,MAX_LEN)
# ,MAX_LEN)

ValueError: ignored

In [ ]:
test_input_ids, test_attention_masks = tokenize(test_data)

In [ ]:
input_test_data={
    'layer_input_ids':test_input_ids,
    'layer_attention_mask':test_attention_masks,
}


In [ ]:
testing_data=" worst day!!!"

In [ ]:
testing_input_ids, testing_attention_masks = tokenize(testing_data)

In [ ]:
input_testing_data={
    'layer_input_ids':np.asarray(testing_input_ids),
    'layer_attention_mask':np.asarray(testing_attention_masks),
}


In [ ]:
predicted_data=model.predict(input_testing_data)


1/1 [==============================] - 0s 184ms/step


In [ ]:
predicted_data

array([[4.6580521e-04, 9.9755812e-01, 1.9761219e-03],
       [1.3529236e-03, 9.9234140e-01, 6.3056899e-03],
       [6.1297785e-03, 9.7998673e-01, 1.3883567e-02],
       [6.1297785e-03, 9.7998673e-01, 1.3883567e-02],
       [1.8034800e-03, 9.9011844e-01, 8.0780396e-03],
       [5.3952625e-03, 9.7372174e-01, 2.0882988e-02],
       [4.7268472e-03, 9.7867018e-01, 1.6602967e-02],
       [3.8281053e-03, 9.7750562e-01, 1.8666320e-02],
       [1.0545816e-03, 9.9642318e-01, 2.5222348e-03],
       [4.6580521e-04, 9.9755812e-01, 1.9761219e-03],
       [5.3952625e-03, 9.7372174e-01, 2.0882988e-02],
       [2.6682923e-03, 9.8502386e-01, 1.2307790e-02],
       [5.3952625e-03, 9.7372174e-01, 2.0882988e-02],
       [7.2180605e-03, 9.6713781e-01, 2.5644166e-02],
       [1.8034800e-03, 9.9011844e-01, 8.0780396e-03],
       [8.3948039e-03, 9.8259830e-01, 9.0069063e-03],
       [1.7884870e-03, 9.9438357e-01, 3.8279267e-03],
       [1.3529236e-03, 9.9234140e-01, 6.3056899e-03],
       [1.0502963e-02, 9.044

In [ ]:
predict_indices=np.argmax(predicted_data,axis=1)

In [ ]:
predicted_data.shape

(29, 3)

In [ ]:
class_labels = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_class_labels = [class_labels[idx] for idx in predict_indices]

In [ ]:
predicted_class_labels

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral']

In [ ]:
test.shape

(3798, 6)

In [ ]:
model.fit(x=input_data,y=y_train,validation_data=(target_data,y_test),epochs=3,verbose=True,batch_size=32)

Epoch 1/3
1029/1029 [==============================] - ETA: 0s - loss: 1.0776 - accuracy: 0.4117

ValueError: ignored

In [ ]:
history_model=model.fit([train_input_ids,train_attention_masks],y_train,validation_data=([val_input_ids,val_attention_masks],y_test),epochs=4,batch_size=32)

Epoch 1/4


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - categorical_accuracy: 1.0000

ValueError: ignored

In [ ]:
train_input=np.asarray([train_input_ids,train_attention_masks])
val_input=np.asarray([val_input_ids,val_attention_masks])